# Homework 3

## Team Members:
### 1. Dongtong Zhong (8124193969)
### 2. Liwen Dai (5282656931)
### 3. Feifan Gu (8135699631)

## Start

In [1]:
# import dataset
import pandas as pd
crsp_dividends = pd.read_csv('CRSP_Dividends_HW3.csv', parse_dates=['date'], index_col='date')
FF_daily = pd.read_csv('FamaFrenchDaily_HW3.csv', parse_dates=['date'], index_col='date')
betas_yearly = pd.read_csv('Yearly_Betas_HW3.csv', parse_dates=['DATE'], index_col='DATE')

## Question 1

In [2]:
# Data cleaning
# A zero trading volume means no trades occurred that day, so the closing price may not be reliable. Such rows are removed to prevent distorted returns.
crsp_dividends = crsp_dividends[crsp_dividends['VOL'] > 0]
# Remove invalid or tiny dividends
crsp_dividends = crsp_dividends[crsp_dividends['DIVAMT'] >= 0.01]
# Set negative prices to their absolute values
crsp_dividends['PRC'] = crsp_dividends['PRC'].abs()
# Remove stocks that have a market capitalization less than $50 million
crsp_dividends['MarketCap'] = crsp_dividends['PRC'] * crsp_dividends['SHROUT'] * 1000
crsp_dividends = crsp_dividends[crsp_dividends['MarketCap'] >= 50e6]

In [3]:
# Data discription
crsp_dividends[['PRC','DIVAMT','MarketCap']].describe()

,PRC,DIVAMT,MarketCap
count,322567.00000,322567.000000,3.225670e+05
mean,37.29405,0.253194,5.238313e+09
std,54.72760,0.501875,2.992906e+10
min,1.01000,0.010000,5.000000e+07
25%,18.71000,0.100000,1.671280e+08
50%,28.20000,0.190000,5.288400e+08
75%,42.87500,0.320000,2.112365e+09
max,4280.04004,85.000000,2.813308e+12


## <mark> Problem solution summary: <mark>

For the above questions in **problem 1**, we first clean the CRSP dividends dataset by removing rows with zero trading volume, filtering out invalid or tiny dividends, converting negative prices to their absolute values, and excluding stocks with a market capitalization below $50 million. After cleaning the data, we provide descriptive statistics for the price (PRC), dividend amount (DIVAMT), and market capitalization (MarketCap) columns to summarize the dataset's characteristics.

## Question 2

In [4]:
# Compute prior day's close using return without dividend (RETX)
crsp_dividends['PriorClose'] = crsp_dividends['PRC'] / (1 + crsp_dividends['RETX'])
# Compute the ratio between prices drop and dividends
crsp_dividends['DropDividendRatio'] = (crsp_dividends['PriorClose'] - crsp_dividends['PRC']) / crsp_dividends['DIVAMT']
# Display summary statistics for DropDividendRatio
crsp_dividends['DropDividendRatio'].describe()

count    322567.000000
mean          0.658522
std          10.621310
min        -906.248481
25%          -0.833365
50%           0.736883
75%           2.272746
max        1071.995035
Name: DropDividendRatio, dtype: float64

### Interpretation in Terms of the Efficient Market Hypothesis (EMH)

When a company pays a dividend ($D$), its stock price should decline by approximately the same amount on the ex-dividend day, since shareholders who purchase the stock afterward are no longer entitled to the dividend. In a perfectly efficient and frictionless market, the expected price drop should therefore be **equal to the dividend**.

The ratio `DropDividendRatio` signifies this relationship:

1. **If the price drop < dividend amount** (`DropDividendRatio < 1`):
   The market does not fully adjust to the dividend payment, implying a potential—but likely short-lived—profit opportunity for dividend-capture strategies.

2. **If the price drop > dividend amount** (`DropDividendRatio > 1`):
   Investors may overreact, or differential tax, liquidity, and timing effects may dominate the price adjustment process.

3. **If the price drop ≈ dividend amount** (`DropDividendRatio = 1`):
   This suggests that the market is efficiently pricing in the dividend payment, consistent with the EMH.

This ratio serves as a **micro-level test of the Efficient Market Hypothesis (EMH)**.
In an efficient market, stock prices should adjust **instantly and rationally** to dividend announcements, leaving no persistent arbitrage opportunities once transaction costs and taxes are considered.

### Interpretation of 'DropDividendRatio' Summary Statistics

- The **mean ratio ($0.66$)** indicates that, on average, stock prices fall by about **$66%$ of the dividend amount** on the ex-dividend day — **less than the full dividend value** predicted by the Efficient Market Hypothesis (EMH).
  This suggests a mild **underreaction**: prices do not drop by the entire dividend amount.

- The **median ratio ($0.74$)** supports this finding, implying that most observations cluster below $1$.

- The **extremely wide range** (from $−906$ to $+1072$) reflects the presence of outliers and thinly traded stocks, where small dividend values can produce large ratios due to price noise or data irregularities. These values should be interpreted cautiously.

A ratio below 1 suggests a potential **dividend-capture investment opportunity** — buying before and selling after the ex-dividend day may yield small short-term profits.

### Constrcut a dividend-capture strategy

In [5]:
import numpy as np
# Compute unconditional (no-look-ahead) dividend-capture return
crsp_dividends['CaptureReturn'] = (
    (crsp_dividends['PRC'] + crsp_dividends['DIVAMT'] - crsp_dividends['PriorClose'])
    / crsp_dividends['PriorClose']
)
# compute marker-cap weighted daily return
daily_ret_vw = (
    crsp_dividends.groupby('date')
    .apply(lambda x: np.average(x['CaptureReturn'], weights=x['MarketCap']))
    .to_frame(name='CaptureReturn')
    .sort_index()
)
daily_ret_vw

,CaptureReturn
date,
1962-01-02,-0.003612
1962-01-03,0.009049
1962-01-04,0.007112
1962-01-05,-0.007543
1962-01-09,0.004611
...,...
2023-06-26,0.014574
2023-06-27,-0.008430
2023-06-28,0.026978


In [6]:
# import risk-free rate data
rf = FF_daily['RF']/ 100  # Convert to decimal
# merge with daily returns
merge_df = pd.merge(daily_ret_vw, rf, left_index=True, right_index=True, how='inner')

# Compute excess return using daily risk-free rate (from Fama-French file)
merge_df['ExcessReturn'] = merge_df['CaptureReturn'] - merge_df['RF']

# Summary statistics
mean_daily = merge_df['CaptureReturn'].mean()
std_daily  = merge_df['CaptureReturn'].std()
mean_excess = merge_df['ExcessReturn'].mean()
std_excess  = merge_df['ExcessReturn'].std()

# Annualized metrics (252 trading days)
annual_return = (1 + mean_daily) ** 252 - 1
annual_vol = std_daily * np.sqrt(252)
sharpe_ratio = (mean_excess / std_excess) * np.sqrt(252)

# Display results
print(f"Annualized Return: {annual_return * 100:.2f}%")
print(f"Annualized Volatility: {annual_vol * 100:.2f}%")
print(f"Sharpe Ratio for Excess Return: {sharpe_ratio:.2f}")

Annualized Return: 34.07%
Annualized Volatility: 21.44%
Sharpe Ratio for Excess Return: 1.17


## <mark> Question solution summary: <mark>

From the above questions in **Question 2**, we first discuss the difference between the price drop and the dividend amount, highlighting how this relates to the Efficient Market Hypothesis (EMH). We then construct a dividend-capture strategy by calculating the unconditional dividend-capture return for each stock on its ex-dividend day. By aggregating these returns using market-cap weighting, we derive a daily portfolio return series. Finally, we compute key performance metrics, including annualized return, annualized volatility, and the Sharpe ratio for the excess return over the risk-free rate, to evaluate the effectiveness of the dividend-capture strategy.

## Question 3

In [18]:
import pandas as pd
import numpy as np

# Load data
df_div = pd.read_csv("CRSP_Dividends_HW3.csv")
ff = pd.read_csv("FamaFrenchDaily_HW3.csv")
betas = pd.read_csv("Yearly_Betas_HW3.csv")

# Date formatting
df_div['date'] = pd.to_datetime(df_div['date'])
ff['date'] = pd.to_datetime(ff['date'].astype(str), format='%Y%m%d')
betas['DATE'] = pd.to_datetime(betas['DATE'])

# Extract year for merging
df_div['year'] = df_div['date'].dt.year
betas['year'] = betas['DATE'].dt.year

# Use previous year's beta to avoid look-ahead bias
betas['year_lag'] = betas['year'] + 1
betas.rename(columns={'b_mkt': 'BETA'}, inplace=True)

df_div = df_div.merge(
    betas[['PERMNO', 'year_lag', 'BETA']],
    how='left',
    left_on=['PERMNO', 'year'],
    right_on=['PERMNO', 'year_lag']
).drop(columns=['year_lag'])

# Merge Fama-French data
df_div = df_div.merge(ff[['date', 'Mkt-RF', 'RF']], on='date', how='left')

# Data cleaning & unit consistency
# 1) Convert CRSP RET to numeric (handle possible 'C' or other string flags)
df_div['RET'] = pd.to_numeric(df_div['RET'], errors='coerce')

# 2) Automatically detect if FF factors are in percent (e.g., 0.10 = 0.10%)
if ff[['Mkt-RF', 'RF']].abs().max().max() > 1:
    df_div['Mkt-RF'] = df_div['Mkt-RF'] / 100.0
    df_div['RF'] = df_div['RF'] / 100.0

# 3) Drop missing values in key columns
df_div.dropna(subset=['BETA', 'Mkt-RF', 'RF', 'RET'], inplace=True)

# CAPM Expected Return and Abnormal Return (log-compounding consistent)
df_div['E_R'] = df_div['RF'] + df_div['BETA'] * df_div['Mkt-RF']

# Keep only valid positive values (required for logarithms)
mask_pos = (1 + df_div['RET'] > 0) & (1 + df_div['E_R'] > 0)
df_div = df_div.loc[mask_pos].copy()

# Compute log returns and log abnormal returns
df_div['logRET'] = np.log1p(df_div['RET'])
df_div['logER']  = np.log1p(df_div['E_R'])
df_div['logAR']  = df_div['logRET'] - df_div['logER']   # daily log abnormal return

# Basic statistics
N = len(df_div)
mean_log_ar = df_div['logAR'].mean()
std_log_ar  = df_div['logAR'].std(ddof=1)
se_log_ar   = std_log_ar / np.sqrt(N)

# t-statistic of the daily mean log abnormal return
t_stat = mean_log_ar / se_log_ar if se_log_ar > 0 else np.nan
sig_5pct = "STATISTICALLY SIGNIFICANT at 5%" if abs(t_stat) >= 1.96 else "NOT statistically significant at 5%"

# Annualization in log space
# Linear scaling in log space
mu_y_log  = 252 * mean_log_ar                  # annualized log mean
sig_y_log = np.sqrt(252) * std_log_ar          # annualized log volatility

# Convert back to arithmetic space for presentation
mean_ar_annual = np.exp(mu_y_log) - 1          # annualized arithmetic mean abnormal return (compounded)
std_ar_annual = np.sqrt((np.exp(sig_y_log**2) - 1) * np.exp(2*mu_y_log + sig_y_log**2))  # annualized arithmetic std from lognormal conversion

# Annualized Sharpe ratios
sharpe_annual_log   = (mean_log_ar / std_log_ar) * np.sqrt(252) if std_log_ar > 0 else np.nan
sharpe_annual_arith = (mean_ar_annual / std_ar_annual) if std_ar_annual > 0 else np.nan

# 95% confidence interval for annualized mean abnormal return
# First compute in log space, then exponentiate back
se_mu_y_log = 252 * se_log_ar
ci_low_annual = np.exp(mu_y_log - 1.96 * se_mu_y_log) - 1
ci_high_annual = np.exp(mu_y_log + 1.96 * se_mu_y_log) - 1

avg_beta = df_div['BETA'].mean()

# Output
print(f"Observations (events): {N:,}")
print(f"Average Beta: {avg_beta:.2f}")
print(f"Annualized Abnormal Return: {mean_ar_annual*100:.2f}%")
print(f"Annualized Std: {std_ar_annual*100:.2f}%")
print(f"95% CI of Annualized Mean AR: [{ci_low_annual*100:.2f}%, {ci_high_annual*100:.2f}%]")
print(f"T-statistic (daily log AR mean): {t_stat:.2f}  => {sig_5pct}")
print(f"Sharpe (annual, log): {sharpe_annual_log:.2f}")
print(f"Sharpe (annual, arithmetic): {sharpe_annual_arith:.2f}")

Observations (events): 392,925
Average Beta: 0.95
Annualized Abnormal Return: 88.12%
Annualized Std: 75.28%
95% CI of Annualized Mean AR: [84.77%, 91.54%]
T-statistic (daily log AR mean): 68.84  => STATISTICALLY SIGNIFICANT at 5%
Sharpe (annual, log): 1.74
Sharpe (annual, arithmetic): 1.17


## <mark> Question solution summary: <mark>

Using CAPM betas and Fama-French factors, the expected return for each dividend event was computed and subtracted from the realized return to get the abnormal return. Based on 392,925 events, the average beta is 0.95. The annualized abnormal return is 88.12% with a volatility of 75.28%, giving a Sharpe ratio of 1.17 and a t-statistic of 68.84, which is statistically significant.

Compared to Question 2 (annualized return = 34.07%, volatility = 21.44%, Sharpe = 1.17), the abnormal return is much higher. This shows that dividend-day price changes generate large and significant excess returns beyond what the CAPM predicts.

## Question 4

In [19]:
import numpy as np
import pandas as pd

# Overnight return construction
# Ensure positive price basis (CRSP PRC can be signed for short-sale convention)
df_div['PRC'] = df_div['PRC'].abs()

# Require open price
df_div = df_div.dropna(subset=['OPENPRC'])

# Prior close from CRSP close-to-close return: RET = (P_t - P_{t-1}) / P_{t-1}
df_div['PriorClose'] = df_div['PRC'] / (1.0 + df_div['RET'])

# Overnight simple return: buy prior close (t-1), sell at open (t)
df_div['overnight_ret'] = (df_div['OPENPRC'] / df_div['PriorClose']) - 1.0

# Keep valid observations for logs
mask = np.isfinite(df_div['overnight_ret']) & (1.0 + df_div['overnight_ret'] > 0)
overnight = df_div.loc[mask, 'overnight_ret'].copy()

# Log-compounding consistent stats
log_r = np.log1p(overnight)

N = log_r.size
mean_log = log_r.mean()
std_log  = log_r.std(ddof=1)
se_log   = std_log / np.sqrt(N)

# t-stat on daily mean (log) return
t_stat = mean_log / se_log if se_log > 0 else np.nan
sig_5pct = "STATISTICALLY SIGNIFICANT at 5%" if np.isfinite(t_stat) and abs(t_stat) >= 1.96 else "NOT statistically significant at 5%"

# Annualize in log space
mu_y_log  = 252 * mean_log
sig_y_log = np.sqrt(252) * std_log

# Convert back to arithmetic for reporting
ann_return = np.exp(mu_y_log) - 1.0
ann_std    = np.sqrt((np.exp(sig_y_log**2) - 1.0) * np.exp(2*mu_y_log + sig_y_log**2))

# Sharpe ratio (annualized): log-based and arithmetic-based
sharpe_annual_log   = (mean_log / std_log) * np.sqrt(252) if std_log > 0 else np.nan
sharpe_annual_arith = (ann_return / ann_std) if ann_std > 0 else np.nan

# Output
print(f"Observations (events): {N:,}")
print(f"Annualized Return: {ann_return*100:.2f}%")
print(f"Annualized Std: {ann_std*100:.2f}%")
print(f"T-statistic (daily mean, log): {t_stat:.2f}  => {sig_5pct}")
print(f"Sharpe Ratio (annual, log): {sharpe_annual_log:.2f}")
print(f"Sharpe Ratio (annual, arithmetic): {sharpe_annual_arith:.2f}")


Observations (events): 178,896
Annualized Return: 56.42%
Annualized Std: 46.35%
T-statistic (daily mean, log): 42.66  => STATISTICALLY SIGNIFICANT at 5%
Sharpe Ratio (annual, log): 1.60
Sharpe Ratio (annual, arithmetic): 1.22


## <mark> Question solution summary: <mark>

For the overnight strategy (buying at the previous day’s close and selling at the next morning’s open), based on 178,896 events, the annualized return is 56.42% with an annualized volatility of 46.35%. The t-statistic of 42.66 indicates the returns are highly statistically significant at the 5% level. The annualized Sharpe ratio is 1.60 using log returns (or 1.22 using arithmetic returns), showing strong risk-adjusted performance.

Overall, this overnight strategy delivers large and statistically significant positive returns, implying that most of the ex-dividend price reaction occurs between the prior close and the next day’s open, rather than during regular trading hours.

## Question 5

In [9]:
# Compute mid-price only once
mid = (crsp_dividends['ASK'] + crsp_dividends['BID']) / 2
# Valid quotes mask: ASK>0, BID>0, ASK>=BID, mid>0
mask = (crsp_dividends['ASK'] > 0) & (crsp_dividends['BID'] > 0) & (crsp_dividends['ASK'] >= crsp_dividends['BID']) & (mid > 0)
# Relative bid–ask spread where quotes are valid; set others to NaN
rel_spread = ((crsp_dividends['ASK'] - crsp_dividends['BID']) / mid).where(mask)
# Transaction cost proxy (round-trip spread): no negatives, fill missing with 0
crsp_dividends['TxnCost'] = rel_spread.clip(lower=0).fillna(0.0)
crsp_dividends['TxnCost'] = crsp_dividends['TxnCost'].clip(upper=crsp_dividends['TxnCost'].quantile(0.995))
crsp_dividends

,PERMNO,SHRCD,PERMCO,DISTCD,DIVAMT,PRC,VOL,RET,BID,ASK,SHROUT,OPENPRC,RETX,vwretd,MarketCap,PriorClose,DropDividendRatio,CaptureReturn,TxnCost
date,,,,,,,,,,,,,,,,,,,
2010-05-12,10001,11,7953,1222,0.045,10.53,10100.0,-0.007974,10.53,10.55,6070.0,10.61,-0.012195,0.016275,6.391710e+07,10.659999,2.888860,-0.007974,0.001898
2010-06-11,10001,11,7953,1222,0.045,11.79,10600.0,-0.002949,11.80,11.82,6071.0,11.89,-0.006740,0.006196,7.157709e+07,11.870004,1.777863,-0.002949,0.001693
2010-07-13,10001,11,7953,1222,0.045,11.00,5100.0,0.004091,10.98,11.04,6080.0,10.98,0.000000,0.017220,6.688000e+07,11.000000,0.000000,0.004091,0.005450
2010-08-11,10001,11,7953,1222,0.045,11.76,9000.0,0.000424,11.76,11.89,6080.0,11.85,-0.003390,-0.029241,7.150080e+07,11.800002,0.888933,0.000424,0.010994
2010-09-13,10001,11,7953,1222,0.045,11.10,5500.0,-0.004911,11.07,11.08,6073.0,11.23,-0.008929,0.012873,6.741030e+07,11.200005,2.222330,-0.004911,0.000903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-27,93429,11,53447,1232,0.480,111.85,491916.0,0.026782,111.62,111.70,106172.0,109.29,0.022395,0.024482,1.187534e+10,109.399987,-5.104193,0.026783,0.000716
2022-08-30,93429,11,53447,1232,0.480,117.78,825426.0,-0.017203,117.94,117.99,106189.0,119.83,-0.021192,-0.011447,1.250694e+10,120.330034,5.312571,-0.017203,0.000424
2022-11-29,93429,11,53447,1232,0.500,123.25,555365.0,-0.007379,123.25,123.27,106062.0,124.44,-0.011390,-0.000744,1.307214e+10,124.669991,2.839982,-0.007379,0.000162


In [10]:
# Calculate net capture return after transaction costs
crsp_dividends['CaptureReturnAfterCost'] = crsp_dividends['CaptureReturn'] - crsp_dividends['TxnCost']
# compute marker-cap weighted daily return after transaction cost
daily_ret_vw_net = (
    crsp_dividends.groupby('date')
    .apply(lambda x: np.average(x['CaptureReturnAfterCost'], weights=x['MarketCap']))
    .to_frame(name='CaptureReturnAfterCost')
    .sort_index()
)

# Calculate annualized return after transaction costs
mean_daily_net = daily_ret_vw_net['CaptureReturnAfterCost'].mean()
annual_return_net = (1 + mean_daily_net) ** 252 - 1
# Display result
print(f"Annualized Return After Transaction Costs: {annual_return_net * 100:.2f}%")

Annualized Return After Transaction Costs: -24.04%


## <mark> Question solution summary: <mark>

In Question 5, transaction costs were incorporated into the dividend-capture strategy by estimating bid-ask spreads as a proxy for trading frictions. After adjusting for these costs, the previously positive gross returns turned significantly negative, showing that transaction expenses more than offset the dividend gains. The results confirm that once realistic market frictions are considered, the dividend-capture strategy provides no excess profit opportunities.

## Question 6

In [11]:
# Read beta data
betas = pd.read_csv('Yearly_Betas_HW3.csv', parse_dates=['DATE'])
# Extract the year of each beta observation (year-end beta)
betas['beta_year'] = betas['DATE'].dt.year
# If multiple rows per stock per year, keep the latest one (usually Dec 31)
betas = betas.sort_values(['PERMNO','DATE']).drop_duplicates(['PERMNO','beta_year'], keep='last')
# Shift the beta year forward by one: use previous-year beta for current-year events
betas['event_year'] = betas['beta_year'] + 1
# Prepare dividend data for merging
df1 = crsp_dividends.reset_index()          
df1['event_year'] = df1['date'].dt.year
# merge beta data
merged_df = df1.merge(betas[['PERMNO','event_year','b_mkt']],how='left',on=['PERMNO','event_year'])
merged_df


,date,PERMNO,SHRCD,PERMCO,DISTCD,DIVAMT,PRC,VOL,RET,BID,...,RETX,vwretd,MarketCap,PriorClose,DropDividendRatio,CaptureReturn,TxnCost,CaptureReturnAfterCost,event_year,b_mkt
0,2010-05-12,10001,11,7953,1222,0.045,10.53,10100.0,-0.007974,10.53,...,-0.012195,0.016275,6.391710e+07,10.659999,2.888860,-0.007974,0.001898,-0.009871,2010,0.2416
1,2010-06-11,10001,11,7953,1222,0.045,11.79,10600.0,-0.002949,11.80,...,-0.006740,0.006196,7.157709e+07,11.870004,1.777863,-0.002949,0.001693,-0.004642,2010,0.2416
2,2010-07-13,10001,11,7953,1222,0.045,11.00,5100.0,0.004091,10.98,...,0.000000,0.017220,6.688000e+07,11.000000,0.000000,0.004091,0.005450,-0.001359,2010,0.2416
3,2010-08-11,10001,11,7953,1222,0.045,11.76,9000.0,0.000424,11.76,...,-0.003390,-0.029241,7.150080e+07,11.800002,0.888933,0.000424,0.010994,-0.010570,2010,0.2416
4,2010-09-13,10001,11,7953,1222,0.045,11.10,5500.0,-0.004911,11.07,...,-0.008929,0.012873,6.741030e+07,11.200005,2.222330,-0.004911,0.000903,-0.005814,2010,0.2416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322562,2022-05-27,93429,11,53447,1232,0.480,111.85,491916.0,0.026782,111.62,...,0.022395,0.024482,1.187534e+10,109.399987,-5.104193,0.026783,0.000716,0.026066,2022,0.6066
322563,2022-08-30,93429,11,53447,1232,0.480,117.78,825426.0,-0.017203,117.94,...,-0.021192,-0.011447,1.250694e+10,120.330034,5.312571,-0.017203,0.000424,-0.017627,2022,0.6066
322564,2022-11-29,93429,11,53447,1232,0.500,123.25,555365.0,-0.007379,123.25,...,-0.011390,-0.000744,1.307214e+10,124.669991,2.839982,-0.007379,0.000162,-0.007542,2022,0.6066
322565,2023-02-27,93429,11,53447,1232,0.500,127.58,577552.0,-0.012414,127.41,...,-0.016270,0.003326,1.353394e+10,129.690057,4.220114,-0.012415,0.001412,-0.013826,2023,0.5459


In [12]:
# Build deciles on MarketCap
s = merged_df['MarketCap'].replace([np.inf, -np.inf], np.nan)
merged_df['MktCap_decile'] = pd.qcut(s, 10, labels=False, duplicates='drop') + 1

# Groupby & aggregate
rnk_mktcap = (
    merged_df.dropna(subset=['MktCap_decile', 'CaptureReturn', 'TxnCost', 'CaptureReturnAfterCost'])
      .groupby('MktCap_decile', observed=True)
      .agg(
          N=('CaptureReturn', 'size'),
          Avg_Return_Daily=('CaptureReturn', 'mean'),
          Avg_Spread_Daily=('TxnCost', 'mean'),
          Avg_NetReturn_Daily=('CaptureReturnAfterCost', 'mean'),
      )
      .sort_index()
)

print(rnk_mktcap)

                   N  Avg_Return_Daily  Avg_Spread_Daily  Avg_NetReturn_Daily
MktCap_decile                                                                
1              32257          0.003399          0.012822            -0.009424
2              32257          0.002997          0.010958            -0.007962
3              32256          0.003109          0.008957            -0.005848
4              32257          0.002404          0.007503            -0.005099
5              32257          0.001969          0.005758            -0.003789
6              32256          0.001905          0.004240            -0.002335
7              32257          0.001448          0.003723            -0.002275
8              32256          0.001380          0.003118            -0.001738
9              32257          0.001370          0.002573            -0.001203
10             32257          0.001020          0.001800            -0.000779


In [13]:
merged_df['DivYield'] = merged_df['DIVAMT'] / merged_df['PriorClose']

# Build deciles on DivYield
m = merged_df['DivYield'].replace([np.inf, -np.inf], np.nan)
merged_df['DivYld_decile'] = pd.qcut(m, 10, labels=False, duplicates='drop') + 1

# Groupby & aggregate
rnk_divyld = (
    merged_df.dropna(subset=['DivYld_decile', 'CaptureReturn', 'TxnCost', 'CaptureReturnAfterCost'])
      .groupby('DivYld_decile', observed=True)
      .agg(
          N=('CaptureReturn', 'size'),
          Avg_Return_Daily=('CaptureReturn', 'mean'),
          Avg_Spread_Daily=('TxnCost', 'mean'),
          Avg_NetReturn_Daily=('CaptureReturnAfterCost', 'mean'),
      )
      .sort_index()
)

print(rnk_divyld)

                   N  Avg_Return_Daily  Avg_Spread_Daily  Avg_NetReturn_Daily
DivYld_decile                                                                
1              32257          0.000413          0.006398            -0.005985
2              32261          0.001415          0.006673            -0.005258
3              32252          0.001746          0.006849            -0.005103
4              32257          0.002102          0.007078            -0.004975
5              32257          0.002600          0.007057            -0.004458
6              32256          0.002826          0.006754            -0.003928
7              32259          0.002980          0.006369            -0.003389
8              32254          0.003081          0.005838            -0.002756
9              32257          0.002971          0.004832            -0.001862
10             32257          0.000868          0.003606            -0.002737


In [14]:
# Build deciles on beta
w = merged_df['b_mkt'].replace([np.inf, -np.inf], np.nan)
merged_df['Beta_decile'] = pd.qcut(w, 10, labels=False, duplicates='drop') + 1

# Groupby & aggregate
rnk_beta = (
    merged_df.dropna(subset=['Beta_decile', 'CaptureReturn', 'TxnCost', 'CaptureReturnAfterCost'])
      .groupby('Beta_decile', observed=True)
      .agg(
          N=('CaptureReturn', 'size'),
          Avg_Return_Daily=('CaptureReturn', 'mean'),
          Avg_Spread_Daily=('TxnCost', 'mean'),
          Avg_NetReturn_Daily=('CaptureReturnAfterCost', 'mean'),
      )
      .sort_index()
)
print(rnk_beta)

                 N  Avg_Return_Daily  Avg_Spread_Daily  Avg_NetReturn_Daily
Beta_decile                                                                
1.0          29637          0.002658          0.011941            -0.009283
2.0          29645          0.001485          0.008772            -0.007287
3.0          29648          0.001622          0.007222            -0.005600
4.0          29630          0.001940          0.006120            -0.004179
5.0          29654          0.001886          0.005299            -0.003414
6.0          29623          0.002255          0.004808            -0.002553
7.0          29624          0.002213          0.004218            -0.002004
8.0          29648          0.002172          0.003501            -0.001330
9.0          29629          0.002273          0.003266            -0.000993
10.0         29628          0.002050          0.003125            -0.001075


## Summary
Sorting the dividend-capture events into deciles by Market Cap, Dividend Yield (D/P), and CAPM Beta (b_mkt) improves gross returns in intuitive ways (small-cap and high-D/P buckets earn higher raw returns; higher beta slightly lowers raw returns). However, bid-ask spreads fall sharply with size and beta and only mildly with D/P, so after subtracting spreads all deciles remain negative on a daily net basis. The best minus worst net improvements are modest and still below zero overall (Market Cap ≈ +0.0086/day; D/P ≈ +0.0032/day; Beta ≈ +0.0082/day), e.g., Market Cap deciles move from about −0.0094 to −0.0008. Conclusion: there is no evidence that decile sorting on these metrics creates a positive net edge for this strategy. If implemented at all, a tilt toward larger-cap / higher-beta with moderate-to-high D/P helps contain costs, but expected net returns remain non-positive; the strategy is not attractive after transaction costs.

## Recommendation
1. If one insists on trading this, **restrict to the most liquid names** (top size deciles, lowest spreads). This **reduces losses** (e.g., D10 net ≈ −0.0008/day vs. D1 ≈ −0.0094/day) but **does not** make average net returns positive.  
2. **Combine** any D/P filter with a **spread screen** (e.g., mid-to-high D/P but **bottom-half spreads**). D/P alone does **not** improve **net** performance.  
3. **Do not** use **beta** sorting to chase returns here; at most, use it to **balance risk exposures**.

## <mark>Question solution summary<mark>:

In Question 6, we sort dividend-capture events into deciles by Market Cap, Dividend Yield (D/P), and CAPM Beta, then calculate each group’s CaptureReturn and adjust for transaction costs using bid-ask spreads to obtain the net return. The results show that small-cap and high D/P portfolios have slightly higher gross returns, but due to larger spreads, all deciles yield negative net returns. Large-cap stocks lose the least, and Beta sorting adds little value. Overall, there is no evidence that decile sorting by these metrics improves strategy profitability after costs.